<a href="https://colab.research.google.com/github/hamsungmin/DataTrainAnalysis/blob/main/project_week2_%EC%97%85%EB%AC%B4%EC%9E%90%EB%8F%99%ED%99%94%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 목표1: 다나와에서 '후드티' 검색 결과를 스크래핑하기

본 파일의 최하단의 '**출력 예시**'와 동일한 출력이 나오게 하는 것을 목표로,
</br>
이하 코드 셀에서 **"코드를 작성해주세요!"** 부분을 적절한 코드로 수정해주세요.
</br>
</br>
※ 단, 다나와의 상품 구성은 언제든 바뀔 수 있으므로, 상품의 상세 내용까지 '출력 예시'와 동일할 필요는 없습니다!

In [ ]:
import requests
from bs4 import BeautifulSoup

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

# 검색어
search_keyword = "후드티"

# 다나와 검색 결과 URL
base_url = f"https://search.danawa.com/dsearch.php?query={search_keyword}"

# HTTP 요청 보내기
response = requests.get(base_url, headers=headers)
response.raise_for_status()  # 요청 실패 시 예외 발생

# HTML 파싱
soup = BeautifulSoup(response.text, 'html.parser')

# 상품명, 가격, 링크, 썸네일 정보 추출
product_elements = soup.select('div.prod_info')
product_prices = soup.select('div.prod_pricelist')
product_images = soup.select('div.thumb_image')


for product, price_element, img in zip(product_elements, product_prices, product_images):
    product_name = product.select("p.prod_name > a")[0].text
    product_price = ""
    product_price_element = price_element.select("p.price_sect > a > strong")
    for index, product_prices in enumerate(product_price_element):
      if index+1 == len(product_price_element):
        product_price += product_prices.text
      else:
        product_price += product_prices.text + ", "
    product_link = img.select('a')[0].get('href')
    # 썸네일 URL 추출
    img_tag = img.find("img")
    _class = img_tag.get('class')
    if (_class is not None) and ("lazyload" in _class):
        product_thumbnail = img_tag.get('data-src')
    else:
        product_thumbnail = img_tag.get('src')

    print(f"상품명: {product_name}, 가격: {product_price}원, 링크: {product_link}, 썸네일: https:{product_thumbnail}")
    # 상세 페이지 접근
    try:
        detail_response = requests.get(product_link, headers=headers)
        detail_response.raise_for_status()
        detail_soup = BeautifulSoup(detail_response.text, 'html.parser')

        # 판매 업체 정보 추출
        seller_elements = detail_soup.select("li.list-item > div.box__logo > img")
        price_elements = detail_soup.select("li.list-item > div.box__price > div.sell-price > span.text__num")
        delivery_elements = detail_soup.select("li.list-item > div.box__delivery")

        for seller, price, delivery in zip(seller_elements, price_elements, delivery_elements):
            seller_name = seller.get('alt')
            seller_price = price.text.strip()
            if len(delivery) > 1:
              seller_delivery = "무료배송"
            else:
              seller_delivery = delivery.text.strip()
            print(f"\t판매 업체: {seller_name}, 가격: {seller_price}원, 배달비: {seller_delivery}")
    except Exception as e:
        print(f"\t판매 업체 정보 추출 중 오류 발생: {e}")

상품명: 디스커버리 익스페디션 덴버 기모 후드티 DXHD73036, 가격: 92,310, 102,480원, 링크: https://prod.danawa.com/info/?pcode=28037165&keyword=%ED%9B%84%EB%93%9C%ED%8B%B0&cate=13230630, 썸네일: https://img.danawa.com/prod_img/500000/165/037/img/28037165_1.jpg?shrink=130:130&_v=20230906134652
	판매 업체: 옥션, 가격: 92,310원, 배달비: 3,000원
	판매 업체: G마켓, 가격: 92,310원, 배달비: 3,000원
상품명: 빈폴 워시드 후드 점퍼 베이지_BC4339O61A, 가격: 150,520원, 링크: https://prod.danawa.com/info/?pcode=78401225&keyword=%ED%9B%84%EB%93%9C%ED%8B%B0&cate=18231859, 썸네일: https://img.danawa.com/prod_img/500000/225/401/img/78401225_1.jpg?shrink=130:130&_v=20250514072447
	판매 업체: GS샵, 가격: 150,520원, 배달비: 무료배송
	판매 업체: 11번가, 가격: 150,530원, 배달비: 무료배송
	판매 업체: GS샵, 가격: 152,030원, 배달비: 무료배송
	판매 업체: 옥션, 가격: 152,040원, 배달비: 무료배송
	판매 업체: G마켓, 가격: 155,950원, 배달비: 무료배송
	판매 업체: 롯데홈쇼핑, 가격: 156,730원, 배달비: 무료배송
	판매 업체: 롯데ON, 가격: 163,125원, 배달비: 무료배송
	판매 업체: SSG.COM, 가격: 171,710원, 배달비: 무료배송
	판매 업체: 신세계몰, 가격: 178,180원, 배달비: 무료배송
	판매 업체: 현대Hmall, 가격: 179,550원, 배달비: 무료배송
상품명: 아디다스 후드 Neuclassics 티셔

# 목표2 (선택): 스크래핑한 정보를 엑셀 파일에 자유로운 형식으로 저장

목표1의 코드를 수정하여, 스크래핑한 정보를 엑셀 파일에 자유로운 형식으로 저장하는 코드를 작성해보세요.
</br>
</br>
※ 목표2는 해보고 싶으신 분들만 하셔도 되는 선택사항입니다!

In [ ]:
import requests
from bs4 import BeautifulSoup
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, Border, Side

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

# 검색어
search_keyword = "후드티"

#엑셀 저장 정보
wb = Workbook()
ws = wb.active
ws.title = "다나와 후트티 상품 정보"

ws['A1'] = "상품명"
ws['B1'] = "최소 가격"
ws['C1'] = "상품 링크"
ws['D1'] = "상품 이미지"
ws['E1'] = "판매 업체"
ws['F1'] = "가격"
ws['G1'] = "배달비"

thick_border = Border(
  left=Side(style="thick"),
  right=Side(style="thick"),
  top=Side(style="thick"),
  bottom=Side(style="thick")
)
ws['A1'].border = thick_border # 굵은 테두리 설정
ws['A1'].font = Font(bold=True)  # 굵은 글씨
ws['A1'].alignment = Alignment(horizontal="center")  # 가운데 정렬

ws['B1'].border = thick_border # 굵은 테두리 설정
ws['B1'].font = Font(bold=True)  # 굵은 글씨
ws['B1'].alignment = Alignment(horizontal="center")  # 가운데 정렬

ws['C1'].border = thick_border # 굵은 테두리 설정
ws['C1'].font = Font(bold=True)  # 굵은 글씨
ws['C1'].alignment = Alignment(horizontal="center")  # 가운데 정렬

ws['D1'].border = thick_border # 굵은 테두리 설정
ws['D1'].font = Font(bold=True)  # 굵은 글씨
ws['D1'].alignment = Alignment(horizontal="center")  # 가운데 정렬

ws['E1'].border = thick_border # 굵은 테두리 설정
ws['E1'].font = Font(bold=True)  # 굵은 글씨
ws['E1'].alignment = Alignment(horizontal="center")  # 가운데 정렬

ws['F1'].border = thick_border # 굵은 테두리 설정
ws['F1'].font = Font(bold=True)  # 굵은 글씨
ws['F1'].alignment = Alignment(horizontal="center")  # 가운데 정렬

ws['G1'].border = thick_border # 굵은 테두리 설정
ws['G1'].font = Font(bold=True)  # 굵은 글씨
ws['G1'].alignment = Alignment(horizontal="center")  # 가운데 정렬

# 다나와 검색 결과 URL
base_url = f"https://search.danawa.com/dsearch.php?query={search_keyword}"

# HTTP 요청 보내기
response = requests.get(base_url, headers=headers)
response.raise_for_status()  # 요청 실패 시 예외 발생

# HTML 파싱
soup = BeautifulSoup(response.text, 'html.parser')

# 상품명, 가격, 링크, 썸네일 정보 추출
product_elements = soup.select('div.prod_info')
product_prices = soup.select('div.prod_pricelist')
product_images = soup.select('div.thumb_image')


for product, price_element, img in zip(product_elements, product_prices, product_images):
    product_name = product.select("p.prod_name > a")[0].text
    product_price = ""
    product_price_element = price_element.select("p.price_sect > a > strong")
    for index, product_prices in enumerate(product_price_element):
      if index+1 == len(product_price_element):
        product_price += product_prices.text
      else:
        product_price += product_prices.text + ", "
    product_link = img.select('a')[0].get('href')
    # 썸네일 URL 추출
    img_tag = img.find("img")
    _class = img_tag.get('class')
    if (_class is not None) and ("lazyload" in _class):
        product_thumbnail = img_tag.get('data-src')
    else:
        product_thumbnail = img_tag.get('src')
    print(f"상품명: {product_name}, 가격: {product_price}원, 링크: {product_link}, 썸네일: https:{product_thumbnail}")
    # 상세 페이지 접근
    try:
        detail_response = requests.get(product_link, headers=headers)
        detail_response.raise_for_status()
        detail_soup = BeautifulSoup(detail_response.text, 'html.parser')

        # 판매 업체 정보 추출
        seller_elements = detail_soup.select("li.list-item > div.box__logo > img")
        price_elements = detail_soup.select("li.list-item > div.box__price > div.sell-price > span.text__num")
        delivery_elements = detail_soup.select("li.list-item > div.box__delivery")

        for index, (seller, price, delivery) in enumerate(zip(seller_elements, price_elements, delivery_elements)):

            seller_name = seller.get('alt')
            seller_price = price.text.strip()
            if len(delivery) > 1:
              seller_delivery = "무료배송"
            else:
              seller_delivery = delivery.text.strip()

            if index == 0:
              ws.append([product_name, product_price, product_link, product_thumbnail, seller_name, seller_price, seller_delivery])
            else:
              ws.append(['', '', '', '', seller_name, seller_price, seller_delivery])
            print(f"\t판매 업체: {seller_name}, 가격: {seller_price}원, 배달비: {seller_delivery}")
    except Exception as e:
        print(f"\t판매 업체 정보 추출 중 오류 발생: {e}")

wb.save('다나와 후드티 상품 정보.xlsx')


상품명: 디스커버리 익스페디션 덴버 기모 후드티 DXHD73036, 가격: 92,310, 102,480원, 링크: https://prod.danawa.com/info/?pcode=28037165&keyword=%ED%9B%84%EB%93%9C%ED%8B%B0&cate=13230630, 썸네일: https://img.danawa.com/prod_img/500000/165/037/img/28037165_1.jpg?shrink=130:130&_v=20230906134652
	판매 업체: 옥션, 가격: 92,310원, 배달비: 3,000원
	판매 업체: G마켓, 가격: 92,310원, 배달비: 3,000원
상품명: 빈폴 워시드 후드 점퍼 베이지_BC4339O61A, 가격: 150,520원, 링크: https://prod.danawa.com/info/?pcode=78401225&keyword=%ED%9B%84%EB%93%9C%ED%8B%B0&cate=18231859, 썸네일: https://img.danawa.com/prod_img/500000/225/401/img/78401225_1.jpg?shrink=130:130&_v=20250514072447
	판매 업체: GS샵, 가격: 150,520원, 배달비: 무료배송
	판매 업체: 11번가, 가격: 150,530원, 배달비: 무료배송
	판매 업체: GS샵, 가격: 152,030원, 배달비: 무료배송
	판매 업체: 옥션, 가격: 152,040원, 배달비: 무료배송
	판매 업체: G마켓, 가격: 155,950원, 배달비: 무료배송
	판매 업체: 롯데홈쇼핑, 가격: 156,730원, 배달비: 무료배송
	판매 업체: 롯데ON, 가격: 163,125원, 배달비: 무료배송
	판매 업체: SSG.COM, 가격: 171,710원, 배달비: 무료배송
	판매 업체: 신세계몰, 가격: 178,180원, 배달비: 무료배송
	판매 업체: 현대Hmall, 가격: 179,550원, 배달비: 무료배송
상품명: 아디다스 후드 Neuclassics 티셔

In [ ]:
raise Exception("출력 예시 셀을 실수로 실행하여 출력 예시가 지워지지 않도록 방지하기 위해 고의적으로 에러를 발생시키는 코드입니다.")

# 출력 예시:

In [ ]:
"이 셀은 출력 예시를 보여주기 위한 셀입니다!"

상품명: 탑텐 밸런스 남성 BALANCE 테크 기모 후디_MBE4TH1260, 가격: 10,450원, 링크: https://prod.danawa.com/info/?pcode=68349017&keyword=%ED%9B%84%EB%93%9C%ED%8B%B0&cate=18231859, 썸네일: https://img.danawa.com/prod_img/500000/017/349/img/68349017_1.jpg?shrink=130:130&_v=20241007113655
	판매 업체: GS샵, 가격: 10,450원, 배달비: 3,000원
	판매 업체: 롯데ON, 가격: 10,680원, 배달비: 3,000원
	판매 업체: 롯데백화점, 가격: 10,680원, 배달비: 3,000원
	판매 업체: 롯데홈쇼핑, 가격: 10,800원, 배달비: 3,000원
	판매 업체: G마켓, 가격: 10,980원, 배달비: 3,000원
	판매 업체: 옥션, 가격: 11,350원, 배달비: 3,000원
	판매 업체: 11번가, 가격: 11,960원, 배달비: 3,000원
	판매 업체: 이랜드몰, 가격: 12,900원, 배달비: 2,500원
	판매 업체: 쿠팡, 가격: 18,400원, 배달비: 무료배송
상품명: 언더아머 UA 라이벌 플리스 후드티 1379861-001, 가격: 32,710원, 링크: https://prod.danawa.com/info/?pcode=32091572&keyword=%ED%9B%84%EB%93%9C%ED%8B%B0&cate=13230630, 썸네일: https://img.danawa.com/prod_img/500000/572/091/img/32091572_1.jpg?shrink=130:130&_v=20240320111306
	판매 업체: 11번가, 가격: 32,710원, 배달비: 3,500원
	판매 업체: G마켓, 가격: 32,720원, 배달비: 3,000원
	판매 업체: 롯데ON, 가격: 32,850원, 배달비: 3,500원
	판매 업체: 옥션, 가격: 33,330원